# Importing the necessary libraries

In [1]:
!pip install datasets transformers rouge-score nltk

import datasets
from datasets import Dataset
from datasets import load_dataset, load_metric

import nltk
nltk.download('punkt')

import os
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.display import display, HTML
import matplotlib.pyplot as plt

import torch
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

import re
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Mounting the google drive with the fine-tuned models

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing the CSV with the test data

In [3]:
test_data = pd.read_csv('/content/drive/MyDrive/NLP/code_review_test_data.csv')
display(HTML(test_data.to_html()))

,text,summary
0,"Hi /r/personalfinance,\n\nI own a condo that I bought in 2008, and initially lived in myself, but have been renting out ever since I moved away in 2010 for a better job. Between the rent, the property management fees, the mortgage payment, PMI, and HOA fees, I estimate that I am paying about $250-300 per month to own this property.\n\nThe tenants in the property for the past couple years are so pleased with it that they're working with a real estate agency (the same I used when buying it) to make me an offer on it. I still don't know the number that they will be offering, but my concern is how to really process that number. There seems to be so much at play:\n\n* Tax breaks I've gotten and continue to get for paying a mortgage\n* Property management fees I've paid over the past 5 years\n* Property tax I've paid over the past 7 years\n* All the mortgage interest that's been paid\n* Taxes that would be assessed as part of the sale\n* 3% agency fees that I'd have to pay for\n* The rent that I didn't have to pay when I was living in the unit, that I would have had to if I hadn't bought it\n\nCan anyone help me figure out how to really quantify the impact that selling it would have? I should also say that I'm in a financially strong position and that owning the property does not put me under any strain. I'd really like to be able to run the numbers and see what it looks like before making a decision one way or the other.\n\nIf it helps to provide specifics about the mortgage and the other costs then say so in the comments and I will.",I own a rental property that I don't really want to be in. Can I sell it? How much can I sell it for? Should I sell it?
1,"My landlord listed my house for sale without my knowledge through a realty agency. He came by a few days later with an addendum to the lease which I refuse to sign. There is a clause in my contract which states ""CHANGE IN LEASE AGREEMENT: The Owner must notify the Occupant of any change and must offer the Occupant a new Agreement or an amendment to the existing Agreement. The Occupant must receive the notice at least 20 days before the proposed effective date of the change. The Occupant may accept the changed terms and conditions by signing the new Agreement and returning it to the Owner or reject the changed terms and conditions by giving the Owner a written 30-day notice terminating its tenancy.\nI told my landlord that he violated the contract by putting the house on the market as of February 1st 2014, then came by the house on February 3 2014 with the addendum, 2 days after he had listed it for sale. I refused to sign the addendum and provided him with a 30-day termination notice. He told that I am interpreting the last sentence of the clause incorrectly. I asked him for his interpretation which he failed to provide. \nDo you see any other interpretation of that sentence ""...or reject the changed terms and conditions by giving the Owner a written 30-day notice terminating its tenancy."". It's pretty damn straightforward from where I'm standing.","Landlord put house on the market as of February 1st 2014. I refused to sign the addendum and he offered to give me a 30-day notice. I told him it was not a valid clause, he said he interpreted it wrong, so I asked for his interpretation of the clause. He failed to provide me with his interpretation. What do I do?"
2,"We both go to the same college and live in the same dorm building. The same hallway to be exact. Lately I've been feeling ""out"" of the the relationship. I feel single but still in a committed relationship if that makes sense. \n\nWe almost broke up last winter but decided not to, and for the most part everything has been fantastic since then, expect the past couple weeks. I've been noticing others girls, getting closer to some friends, and just feeling like I might have outgrown my relationship. \n\nI know that if/when I break up with her it's totally going to blindside her and that hurts, because I s

# Data preprocessing

In [4]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                       "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                       "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                       "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have",
                       "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                       "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                       "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                       "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                       "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
                       "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                       "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                       "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                       "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                       "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are",
                       "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have",
                       "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
                       "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have",
                       "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have",
                       "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is",
                       "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
                       "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would",
                       "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [5]:
def text_cleaner(text):
    string = text.lower()
    string = BeautifulSoup(string, "lxml").text
    string = re.sub(r'', '', string)
    string = re.sub('"', '', string)
    string = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in string.split(" ")])
    string = re.sub(r"'s\b", "", string)
    string = re.sub(r'\.{2,}', ' ', string)
    string = re.sub("[^a-zA-Z.]", " ", string)
    string = re.sub(r'\s+', ' ', string).strip()

    return string

In [6]:
test_data['text'] = test_data['text'].map(text_cleaner)
test_data['summary'] = test_data['summary'].map(text_cleaner)

# Data after preprocessing

In [7]:
display(HTML(test_data.to_html()))

,text,summary
0,hi r personalfinance i own a condo that i bought in and initially lived in myself but have been renting out ever since i moved away in for a better job. between the rent the property management fees the mortgage payment pmi and hoa fees i estimate that i am paying about per month to own this property. the tenants in the property for the past couple years are so pleased with it that they are working with a real estate agency the same i used when buying it to make me an offer on it. i still do not know the number that they will be offering but my concern is how to really process that number. there seems to be so much at play tax breaks i have gotten and continue to get for paying a mortgage property management fees i have paid over the past years property tax i have paid over the past years all the mortgage interest that is been paid taxes that would be assessed as part of the sale agency fees that i would have to pay for the rent that i did not have to pay when i was living in the unit that i would have had to if i had not bought it can anyone help me figure out how to really quantify the impact that selling it would have i should also say that i am in a financially strong position and that owning the property does not put me under any strain. i would really like to be able to run the numbers and see what it looks like before making a decision one way or the other. if it helps to provide specifics about the mortgage and the other costs then say so in the comments and i will.,i own a rental property that i do not really want to be in. can i sell it how much can i sell it for should i sell it
1,my landlord listed my house for sale without my knowledge through a realty agency. he came by a few days later with an addendum to the lease which i refuse to sign. there is a clause in my contract which states change in lease agreement the owner must notify the occupant of any change and must offer the occupant a new agreement or an amendment to the existing agreement. the occupant must receive the notice at least days before the proposed effective date of the change. the occupant may accept the changed terms and conditions by signing the new agreement and returning it to the owner or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy. i told my landlord that he violated the contract by putting the house on the market as of february st then came by the house on february with the addendum days after he had listed it for sale. i refused to sign the addendum and provided him with a day termination notice. he told that i am interpreting the last sentence of the clause incorrectly. i asked him for his interpretation which he failed to provide. do you see any other interpretation of that sentence or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy it is pretty damn straightforward from where i am standing.,landlord put house on the market as of february st . i refused to sign the addendum and he offered to give me a day notice. i told him it was not a valid clause he said he interpreted it wrong so i asked for his interpretation of the clause. he failed to provide me with his interpretation. what do i do
2,we both go to the same college and live in the same dorm building. the same hallway to be exact. lately i have been feeling out of the the relationship. i feel single but still in a committed relationship if that makes sense. we almost broke up last winter but decided not to and for the most part everything has been fantastic since then expect the past couple weeks. i have been noticing others girls getting closer to some friends and just feeling like i might have outgrown my relationship. i know that if when i break up with her it is totally going to blindside her and that hurts because i still love and care for her. i just do not see a future in this relationship anymore. so i guess i have a couple questions. do you 

# Loading the evaluation metric

In [8]:
metric = load_metric("rouge")

<ipython-input-8-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


# Function to generate dataframe with summaries and ROUGE scores

In [9]:
def generate_summary(path, model_tokenizer):
  model_loaded = AutoModelForSeq2SeqLM.from_pretrained(PATH)
  summarizer = pipeline("summarization", model = model_loaded, tokenizer = tokenizer, device = 0)

  text_test = test_data.text.tolist()
  summary_test = test_data.summary.tolist()
  summary_pred = []

  batch_size = 8
  start_idx = 0

  for i in range(0, len(text_test[start_idx:]), batch_size):
      batch = int(i/batch_size)
      batch_text = text_test[start_idx+i : start_idx+i+batch_size]
      summary_pred += [elem['summary_text'] for elem in summarizer(batch_text, min_length = 10, max_length = 50)]
      if batch == 2:
          break

  rouge1 = []
  rouge2 = []
  rougeL = []

  for i in range(len(summary_pred)):
      results = metric.compute(predictions = [summary_pred[i]], references = [summary_test[i]])
      for key in ["rouge1", "rouge2", "rougeL"]:
          if key == "rouge1":
              rouge1.append(results[key].mid.fmeasure)
          elif key == "rouge2":
              rouge2.append(results[key].mid.fmeasure)
          else:
              rougeL.append(results[key].mid.fmeasure)

  n = len(summary_pred)
  df_results_sample = pd.DataFrame({"text": text_test[:n],
                                  "summary": summary_test[:n],
                                  "summary_pred": summary_pred[:n],
                                  "rouge1": rouge1,
                                  "rouge2": rouge2,
                                  "rougeL": rougeL}
                                  )

  return df_results_sample

# BART-base

In [10]:
model = 'facebook/bart-base'
tokenizer = AutoTokenizer.from_pretrained(model)

PATH = '/content/drive/MyDrive/NLP/BART-base'

model_loaded = AutoModelForSeq2SeqLM.from_pretrained(PATH)
summarizer = pipeline("summarization", model = model_loaded, tokenizer = tokenizer, device = 0)

BART_base_df = generate_summary(PATH, tokenizer)

In [11]:
display(HTML(BART_base_df.to_html()))

,text,summary,summary_pred,rouge1,rouge2,rougeL
0,hi r personalfinance i own a condo that i bought in and initially lived in myself but have been renting out ever since i moved away in for a better job. between the rent the property management fees the mortgage payment pmi and hoa fees i estimate that i am paying about per month to own this property. the tenants in the property for the past couple years are so pleased with it that they are working with a real estate agency the same i used when buying it to make me an offer on it. i still do not know the number that they will be offering but my concern is how to really process that number. there seems to be so much at play tax breaks i have gotten and continue to get for paying a mortgage property management fees i have paid over the past years property tax i have paid over the past years all the mortgage interest that is been paid taxes that would be assessed as part of the sale agency fees that i would have to pay for the rent that i did not have to pay when i was living in the unit that i would have had to if i had not bought it can anyone help me figure out how to really quantify the impact that selling it would have i should also say that i am in a financially strong position and that owning the property does not put me under any strain. i would really like to be able to run the numbers and see what it looks like before making a decision one way or the other. if it helps to provide specifics about the mortgage and the other costs then say so in the comments and i will.,i own a rental property that i do not really want to be in. can i sell it how much can i sell it for should i sell it,i own a condo that i bought in and initially lived in myself but have been renting out ever since i moved away in for a better job. between the rent the property management fees the mortgage payment pmi and hoa fees i estimate,0.273973,0.084507,0.246575
1,my landlord listed my house for sale without my knowledge through a realty agency. he came by a few days later with an addendum to the lease which i refuse to sign. there is a clause in my contract which states change in lease agreement the owner must notify the occupant of any change and must offer the occupant a new agreement or an amendment to the existing agreement. the occupant must receive the notice at least days before the proposed effective date of the change. the occupant may accept the changed terms and conditions by signing the new agreement and returning it to the owner or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy. i told my landlord that he violated the contract by putting the house on the market as of february st then came by the house on february with the addendum days after he had listed it for sale. i refused to sign the addendum and provided him with a day termination notice. he told that i am interpreting the last sentence of the clause incorrectly. i asked him for his interpretation which he failed to provide. do you see any other interpretation of that sentence or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy it is pretty damn straightforward from where i am standing.,landlord put house on the market as of february st . i refused to sign the addendum and he offered to give me a day notice. i told him it was not a valid clause he said he interpreted it wrong so i asked for his interpretation of the clause. he failed to provide me with his interpretation. what do i do,landlord listed my house for sale without my knowledge and gave the owner a written day notice terminating its tenancy. do you see any other interpretation of the clause,0.288889,0.090909,0.222222
2,we both go to the same college and live in the same dorm building. the same hallway to be exact. lately i have been feeling out of the the relationship. i feel single but still in a committed relationship if that makes sense. we almost broke u

# BART-large

In [12]:
model = 'facebook/bart-large'
tokenizer = AutoTokenizer.from_pretrained(model)

PATH = '/content/drive/MyDrive/NLP/BART-large'

model_loaded = AutoModelForSeq2SeqLM.from_pretrained(PATH)
summarizer = pipeline("summarization", model = model_loaded, tokenizer = tokenizer, device = 0)

BART_large_df = generate_summary(PATH, tokenizer)

In [13]:
display(HTML(BART_large_df.to_html()))

,text,summary,summary_pred,rouge1,rouge2,rougeL
0,hi r personalfinance i own a condo that i bought in and initially lived in myself but have been renting out ever since i moved away in for a better job. between the rent the property management fees the mortgage payment pmi and hoa fees i estimate that i am paying about per month to own this property. the tenants in the property for the past couple years are so pleased with it that they are working with a real estate agency the same i used when buying it to make me an offer on it. i still do not know the number that they will be offering but my concern is how to really process that number. there seems to be so much at play tax breaks i have gotten and continue to get for paying a mortgage property management fees i have paid over the past years property tax i have paid over the past years all the mortgage interest that is been paid taxes that would be assessed as part of the sale agency fees that i would have to pay for the rent that i did not have to pay when i was living in the unit that i would have had to if i had not bought it can anyone help me figure out how to really quantify the impact that selling it would have i should also say that i am in a financially strong position and that owning the property does not put me under any strain. i would really like to be able to run the numbers and see what it looks like before making a decision one way or the other. if it helps to provide specifics about the mortgage and the other costs then say so in the comments and i will.,i own a rental property that i do not really want to be in. can i sell it how much can i sell it for should i sell it,i own a condo that i bought in and initially lived in myself but have been renting out ever since i moved away in for a better job. the tenants in the property are so pleased with it that they are working with a real estate,0.266667,0.082192,0.240000
1,my landlord listed my house for sale without my knowledge through a realty agency. he came by a few days later with an addendum to the lease which i refuse to sign. there is a clause in my contract which states change in lease agreement the owner must notify the occupant of any change and must offer the occupant a new agreement or an amendment to the existing agreement. the occupant must receive the notice at least days before the proposed effective date of the change. the occupant may accept the changed terms and conditions by signing the new agreement and returning it to the owner or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy. i told my landlord that he violated the contract by putting the house on the market as of february st then came by the house on february with the addendum days after he had listed it for sale. i refused to sign the addendum and provided him with a day termination notice. he told that i am interpreting the last sentence of the clause incorrectly. i asked him for his interpretation which he failed to provide. do you see any other interpretation of that sentence or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy it is pretty damn straightforward from where i am standing.,landlord put house on the market as of february st . i refused to sign the addendum and he offered to give me a day notice. i told him it was not a valid clause he said he interpreted it wrong so i asked for his interpretation of the clause. he failed to provide me with his interpretation. what do i do,landlord put my house on the market without my knowledge. i refuse to sign an addendum to the lease. he then came by with an additional addendum days after he had listed the house for sale. i refused to sign the,0.392157,0.160000,0.294118
2,we both go to the same college and live in the same dorm building. the same hallway to be exact. lately i have been feeling out of the the relationship. i feel single but still in a committed relationship

# BART-large-cnn

In [14]:
model = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model)

PATH = '/content/drive/MyDrive/NLP/BART-large-cnn'

model_loaded = AutoModelForSeq2SeqLM.from_pretrained(PATH)
summarizer = pipeline("summarization", model = model_loaded, tokenizer = tokenizer, device = 0)

BART_large_cnn_df = generate_summary(PATH, tokenizer)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
display(HTML(BART_large_cnn_df.to_html()))

,text,summary,summary_pred,rouge1,rouge2,rougeL
0,hi r personalfinance i own a condo that i bought in and initially lived in myself but have been renting out ever since i moved away in for a better job. between the rent the property management fees the mortgage payment pmi and hoa fees i estimate that i am paying about per month to own this property. the tenants in the property for the past couple years are so pleased with it that they are working with a real estate agency the same i used when buying it to make me an offer on it. i still do not know the number that they will be offering but my concern is how to really process that number. there seems to be so much at play tax breaks i have gotten and continue to get for paying a mortgage property management fees i have paid over the past years property tax i have paid over the past years all the mortgage interest that is been paid taxes that would be assessed as part of the sale agency fees that i would have to pay for the rent that i did not have to pay when i was living in the unit that i would have had to if i had not bought it can anyone help me figure out how to really quantify the impact that selling it would have i should also say that i am in a financially strong position and that owning the property does not put me under any strain. i would really like to be able to run the numbers and see what it looks like before making a decision one way or the other. if it helps to provide specifics about the mortgage and the other costs then say so in the comments and i will.,i own a rental property that i do not really want to be in. can i sell it how much can i sell it for should i sell it,i own a rental property that i have been renting out for the past couple years. i am thinking about selling it but i do not know how to really figure out the impact of selling it.,0.515152,0.250000,0.333333
1,my landlord listed my house for sale without my knowledge through a realty agency. he came by a few days later with an addendum to the lease which i refuse to sign. there is a clause in my contract which states change in lease agreement the owner must notify the occupant of any change and must offer the occupant a new agreement or an amendment to the existing agreement. the occupant must receive the notice at least days before the proposed effective date of the change. the occupant may accept the changed terms and conditions by signing the new agreement and returning it to the owner or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy. i told my landlord that he violated the contract by putting the house on the market as of february st then came by the house on february with the addendum days after he had listed it for sale. i refused to sign the addendum and provided him with a day termination notice. he told that i am interpreting the last sentence of the clause incorrectly. i asked him for his interpretation which he failed to provide. do you see any other interpretation of that sentence or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy it is pretty damn straightforward from where i am standing.,landlord put house on the market as of february st . i refused to sign the addendum and he offered to give me a day notice. i told him it was not a valid clause he said he interpreted it wrong so i asked for his interpretation of the clause. he failed to provide me with his interpretation. what do i do,landlord violated contract by putting house on the market as of february st then came by with addendum days after he had listed it for sale. interpretation of clause saying owner must notify the occupant of any change,0.363636,0.164948,0.323232
2,we both go to the same college and live in the same dorm building. the same hallway to be exact. lately i have been feeling out of the the relationship. i feel single but still in a committed relationship if that makes sense. we almost broke 

# T5-base

In [16]:
model = 't5-base'
tokenizer = AutoTokenizer.from_pretrained(model)

PATH = '/content/drive/MyDrive/NLP/T5-base'

model_loaded = AutoModelForSeq2SeqLM.from_pretrained(PATH)
summarizer = pipeline("summarization", model = model_loaded, tokenizer = tokenizer, device = 0)

T5_base_df = generate_summary(PATH, tokenizer)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [17]:
display(HTML(T5_base_df.to_html()))

,text,summary,summary_pred,rouge1,rouge2,rougeL
0,hi r personalfinance i own a condo that i bought in and initially lived in myself but have been renting out ever since i moved away in for a better job. between the rent the property management fees the mortgage payment pmi and hoa fees i estimate that i am paying about per month to own this property. the tenants in the property for the past couple years are so pleased with it that they are working with a real estate agency the same i used when buying it to make me an offer on it. i still do not know the number that they will be offering but my concern is how to really process that number. there seems to be so much at play tax breaks i have gotten and continue to get for paying a mortgage property management fees i have paid over the past years property tax i have paid over the past years all the mortgage interest that is been paid taxes that would be assessed as part of the sale agency fees that i would have to pay for the rent that i did not have to pay when i was living in the unit that i would have had to if i had not bought it can anyone help me figure out how to really quantify the impact that selling it would have i should also say that i am in a financially strong position and that owning the property does not put me under any strain. i would really like to be able to run the numbers and see what it looks like before making a decision one way or the other. if it helps to provide specifics about the mortgage and the other costs then say so in the comments and i will.,i own a rental property that i do not really want to be in. can i sell it how much can i sell it for should i sell it,i own a condo that i bought in and have been renting out ever since i moved away in for a better job. i do not know how to quantify the impact that selling it would have on me.,0.411765,0.151515,0.294118
1,my landlord listed my house for sale without my knowledge through a realty agency. he came by a few days later with an addendum to the lease which i refuse to sign. there is a clause in my contract which states change in lease agreement the owner must notify the occupant of any change and must offer the occupant a new agreement or an amendment to the existing agreement. the occupant must receive the notice at least days before the proposed effective date of the change. the occupant may accept the changed terms and conditions by signing the new agreement and returning it to the owner or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy. i told my landlord that he violated the contract by putting the house on the market as of february st then came by the house on february with the addendum days after he had listed it for sale. i refused to sign the addendum and provided him with a day termination notice. he told that i am interpreting the last sentence of the clause incorrectly. i asked him for his interpretation which he failed to provide. do you see any other interpretation of that sentence or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy it is pretty damn straightforward from where i am standing.,landlord put house on the market as of february st . i refused to sign the addendum and he offered to give me a day notice. i told him it was not a valid clause he said he interpreted it wrong so i asked for his interpretation of the clause. he failed to provide me with his interpretation. what do i do,my landlord listed my house for sale without my knowledge. he came by the house days later with an addendum to the lease which i refuse to sign. i refused to sign and gave him a day termination notice.,0.400000,0.081633,0.220000
2,we both go to the same college and live in the same dorm building. the same hallway to be exact. lately i have been feeling out of the the relationship. i feel single but still in a committed relationship if that makes sense. we almost broke up last winter but 

# T5-small

In [18]:
model = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model)

PATH = '/content/drive/MyDrive/NLP/T5-base'

model_loaded = AutoModelForSeq2SeqLM.from_pretrained(PATH)
summarizer = pipeline("summarization", model = model_loaded, tokenizer = tokenizer, device = 0)

T5_small_df = generate_summary(PATH, tokenizer)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [19]:
display(HTML(T5_small_df.to_html()))

,text,summary,summary_pred,rouge1,rouge2,rougeL
0,hi r personalfinance i own a condo that i bought in and initially lived in myself but have been renting out ever since i moved away in for a better job. between the rent the property management fees the mortgage payment pmi and hoa fees i estimate that i am paying about per month to own this property. the tenants in the property for the past couple years are so pleased with it that they are working with a real estate agency the same i used when buying it to make me an offer on it. i still do not know the number that they will be offering but my concern is how to really process that number. there seems to be so much at play tax breaks i have gotten and continue to get for paying a mortgage property management fees i have paid over the past years property tax i have paid over the past years all the mortgage interest that is been paid taxes that would be assessed as part of the sale agency fees that i would have to pay for the rent that i did not have to pay when i was living in the unit that i would have had to if i had not bought it can anyone help me figure out how to really quantify the impact that selling it would have i should also say that i am in a financially strong position and that owning the property does not put me under any strain. i would really like to be able to run the numbers and see what it looks like before making a decision one way or the other. if it helps to provide specifics about the mortgage and the other costs then say so in the comments and i will.,i own a rental property that i do not really want to be in. can i sell it how much can i sell it for should i sell it,i own a condo that i bought in and have been renting out ever since i moved away in for a better job. i do not know how to quantify the impact that selling it would have on me.,0.411765,0.151515,0.294118
1,my landlord listed my house for sale without my knowledge through a realty agency. he came by a few days later with an addendum to the lease which i refuse to sign. there is a clause in my contract which states change in lease agreement the owner must notify the occupant of any change and must offer the occupant a new agreement or an amendment to the existing agreement. the occupant must receive the notice at least days before the proposed effective date of the change. the occupant may accept the changed terms and conditions by signing the new agreement and returning it to the owner or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy. i told my landlord that he violated the contract by putting the house on the market as of february st then came by the house on february with the addendum days after he had listed it for sale. i refused to sign the addendum and provided him with a day termination notice. he told that i am interpreting the last sentence of the clause incorrectly. i asked him for his interpretation which he failed to provide. do you see any other interpretation of that sentence or reject the changed terms and conditions by giving the owner a written day notice terminating its tenancy it is pretty damn straightforward from where i am standing.,landlord put house on the market as of february st . i refused to sign the addendum and he offered to give me a day notice. i told him it was not a valid clause he said he interpreted it wrong so i asked for his interpretation of the clause. he failed to provide me with his interpretation. what do i do,my landlord listed my house for sale without my knowledge. he came by the house days later with an addendum to the lease which i refuse to sign. i refused to sign and gave him a day termination notice.,0.400000,0.081633,0.220000
2,we both go to the same college and live in the same dorm building. the same hallway to be exact. lately i have been feeling out of the the relationship. i feel single but still in a committed relationship if that makes sense. we almost broke up last winter but 